## Demo of Feature Extraction Pipeline and its Intended Usage

##### Please ensure that you have spaCy and the "es_core_news_md" pipeline installed

In [1]:
# !python -m spacy download es_core_news_md

In [2]:
import pprint
from utils import read_corpus
from features import feature_pipeline

# Setup pretty printer
p = pprint.PrettyPrinter(indent=4, width=140)

### Corpus Reading
Let's see what the corpus looks like currently:

In [3]:
corpus = read_corpus()

for k, v in corpus.items():
    print(f"{k}: {len(v)}")

A1: 94
A2: 62
B2: 110
B1: 42


Let's pick a text from the corpus for the purposes of this demo:

In [4]:
un_processed_text = corpus["A1"][80]["content"]
print(un_processed_text)

15. EL INVIERNO
El invierno es la estación fría. En el invierno
los días son muy cortos y las noches son
muy largas. Cuando hace mucho frío el agua
se hiela y cae nieve. En la zona tórrida no hay
hielo ni nieve, y hace siempre calor. En las
zonas templadas no hay hielo ni nieve sino en
el invierno. En las zonas glaciales hay hielo y
nieve durante todas las estaciones.
Los muchachos están alegres cuando hiela y
cae la nieve. Entonces patinan en los ríos y en
los lagos helados. Hacen pelotas de nieve y se
las arrojan unos a otros y juegan a la guerra.
Hacen también imágenes de nieve. Cuando ha
caído la nieve los muchachos traen sus trineos
sobre la nieve.
Cuando deshiela y la nieve desaparece, los
niños están muy tristes porque no pueden
patinar ni jugar más con pelotas de nieve.
Pero los pobres están muy alegres, porque
durante el invierno hace demasiado frío para
ellos. En la primavera ellos no tienen frío.
El invierno dura desde el veintiuno de diciembre
hasta el veintiuno de marzo.



We can see that this piece of text is formatted somewhat "strangely". It is a poem, so it has line breaks in the middle of sentences. It also has a title at the top, which is not a necessary component of the content of the text.

### Text Preprocessing
Let's create a pipeline for cleaning up this text and extracting important attributes and features from it.

In [5]:
# This step passes the un-processed text to the pipeline and automatically cleans it up using the .preprocess() method
pipe = feature_pipeline(un_processed_text)
print(pipe.text)

el invierno el invierno es la estación fría. en el invierno los días son muy cortos y las noches son muy largas. cuando hace mucho frío el agua se hiela y cae nieve. en la zona tórrida no hay hielo ni nieve, y hace siempre calor. en las zonas templadas no hay hielo ni nieve sino en el invierno. en las zonas glaciales hay hielo y nieve durante todas las estaciones. los muchachos están alegres cuando hiela y cae la nieve. entonces patinan en los ríos y en los lagos helados. hacen pelotas de nieve y se las arrojan unos a otros y juegan a la guerra. hacen también imágenes de nieve. cuando ha caído la nieve los muchachos traen sus trineos sobre la nieve. cuando deshiela y la nieve desaparece, los niños están muy tristes porque no pueden patinar ni jugar más con pelotas de nieve. pero los pobres están muy alegres, porque durante el invierno hace demasiado frío para ellos. en la primavera ellos no tienen frío. el invierno dura desde el veintiuno de diciembre hasta el veintiuno de marzo.


The text looks much more standardized now, which makes it easier for downstream functions to extract things from it in a consistent manner.

In [6]:
# NOTE: alternatively we could achieve the above by creating a blank pipeline object
pipe = feature_pipeline()
# and manually call
cleaned_text = pipe.preprocess(un_processed_text)
# to clean up the text

### Extracting Attributes of the Text using SpaCy
Some attributes of the text that we might be interested in are the sentences, tokens and POS tags. We can try accessing them, but they won't be accessible at this stage.

In [7]:
# The below calls to access class attributes will just return empty lists since those items have not been extracted yet
print(pipe.sentences)
print(pipe.tokens)
print(pipe.pos_tags)

[]
[]
[]


These attributes must be extracted from the text using spaCy's Spanish pipeline. It is recommended that we generate the lists as and when we need them, since extracting all of them for each text can be a bit slow (although the text preprocessing is typically the slowest step in our pipeline). We can extract some attributes as follows:

In [8]:
p.pprint(pipe.get_sentences())  # populates the pipe.sentences attribute
print()
print(pipe.get_tokens())  # populates the pipe.tokens attribute
# etc...

[   'el invierno el invierno es la estación fría.',
    'en el invierno los días son muy cortos y las noches son muy largas.',
    'cuando hace mucho frío el agua se hiela y cae nieve.',
    'en la zona tórrida no hay hielo ni nieve, y hace siempre calor.',
    'en las zonas templadas no hay hielo ni nieve sino en el invierno.',
    'en las zonas glaciales hay hielo y nieve durante todas las estaciones.',
    'los muchachos están alegres cuando hiela y cae la nieve.',
    'entonces patinan en los ríos y en los lagos helados.',
    'hacen pelotas de nieve y se las arrojan unos a otros y juegan a la guerra.',
    'hacen también imágenes de nieve.',
    'cuando ha caído la nieve los muchachos traen sus trineos sobre la nieve.',
    'cuando deshiela y la nieve desaparece, los niños están muy tristes porque no pueden patinar ni jugar más con pelotas de nieve.',
    'pero los pobres están muy alegres, porque durante el invierno hace demasiado frío para ellos.',
    'en la primavera ellos no 

In [9]:
# These function calls also return the list object directly,
#  so we could optionally just assign them to a variable:
tags = pipe.get_pos_tags()
# pipe.pos_tags will still get populated and can be accessed directly that way as well

In [10]:
# NOTE: alternatively we could create a blank pipeline object
pipe = feature_pipeline()
# and manually call
tags = pipe.get_pos_tags(un_processed_text)
# to automatically clean up the text AS WELL AS extract the POS tags

\
Here's a list of all of the spaCy features supported by our pipeline so far:
* sentences: \
    extraction function - `pipe.get_sentences()`, \
    attribute - `pipe.sentences`
* tokens: \
    extraction function - `pipe.get_tokens()`, \
    attribute - `pipe.tokens`
* lemmas: \
    extraction function - `pipe.get_lemmas()`, attribute - `pipe.lemmas`
* POS tags: \
    extraction function - `pipe.get_pos_tags()`, \
    attribute - `pipe.pos_tags`
* morphology tags: \
    extraction function - `pipe.get_morphology()`, \
    attribute - `pipe.morphs`
* dependency parses: \
    extraction function - `pipe.get_dependency_parses()`, \
    attribute - `pipe.parses`
* noun phrase chunks: \
    extraction function - `pipe.get_noun_chunks()`, \
    attribute - `pipe.noun_chunks`

What if you want to extract all of the spaCy features in one go, instead of calling each of the `.get_*` methods one by one? You can do that by calling the method `.full_spacy()` which will extract all of these features, OR you could initialize the pipeline object with the flag `full_spacy=True`.

In [11]:
pipe = feature_pipeline(un_processed_text, full_spacy=True)
# Equivalent to calling pipe.full_spacy(un_processed_text)

# All of the following items will automatically be extracted as part of the spaCy pipeline:
print(pipe.text)
print()
print(pipe.noun_chunks)

# Commented out for brevity
# p.pprint(pipe.sentences)
# print(pipe.tokens)
# print(pipe.lemmas)
# print(pipe.pos_tags)
# print(pipe.parses)
# print(pipe.morphs)

el invierno el invierno es la estación fría. en el invierno los días son muy cortos y las noches son muy largas. cuando hace mucho frío el agua se hiela y cae nieve. en la zona tórrida no hay hielo ni nieve, y hace siempre calor. en las zonas templadas no hay hielo ni nieve sino en el invierno. en las zonas glaciales hay hielo y nieve durante todas las estaciones. los muchachos están alegres cuando hiela y cae la nieve. entonces patinan en los ríos y en los lagos helados. hacen pelotas de nieve y se las arrojan unos a otros y juegan a la guerra. hacen también imágenes de nieve. cuando ha caído la nieve los muchachos traen sus trineos sobre la nieve. cuando deshiela y la nieve desaparece, los niños están muy tristes porque no pueden patinar ni jugar más con pelotas de nieve. pero los pobres están muy alegres, porque durante el invierno hace demasiado frío para ellos. en la primavera ellos no tienen frío. el invierno dura desde el veintiuno de diciembre hasta el veintiuno de marzo.

['el

### Numerical/Statistical Feature Extraction
The most important aspect of the feature extraction pipeline is the ability to extract statistical/numerical features from the text given to it. For a comprehensive guide of all of the features that this pipeline is capable of extracting please see the project report (TODO: LINK TO PROJECT REPORT). \
\
Here is how we can extract some features from a text using the pipeline:

In [12]:
num_tokens = pipe.num_tokens()
log_op_density = pipe.logical_operators()
fh_score, syls_per_sent = pipe.fernandez_huerta_score()

print(num_tokens)
print(log_op_density)
print(fh_score)
print(syls_per_sent)

200
0.06951871657754011
90.29653846153847
21.866666666666667


Note that any of the statistical feature functions can be called directly without needing to run any of the spaCy extractors first. As long as a feature pipeline object has been created with a text associated with it, calling any of the feature functions will automatically extract the spaCy features necessary for computing the desired statistical feature. For example:

In [13]:
# Explicitly specifying full_spacy=False for demonstration purposes
pipe = feature_pipeline(un_processed_text, full_spacy=False)
len_without_stopwords, a_level_token_prop, a_level_type_prop = pipe.a_level_vocab_features()

print(len_without_stopwords)
print(a_level_token_prop)
print(a_level_type_prop)

82
0.5
0.3617021276595745


Finally, let's extract all of the available statistical features in one go. That is as simple as creating a pipeline object and calling the `.feature_extractor()` method. \
(NOTE: If the object is initialized with a text, `feature_extractor` does not require any arguments. Otherwise, if the object is initialized *without* a text, `feature_extractor` must be called with a text in order to perform preprocessing and spaCy attribute extraction.)

In [14]:
pipe = feature_pipeline()
p.pprint(pipe.feature_extractor(un_processed_text))

# ALTERNATIVELY:
# pipe = feature_pipeline(un_processed_text)
# pipe.feature_extractor()

{   'ADJ': 0.065,
    'ADP': 0.11,
    'ADV': 0.065,
    'AUX': 0.075,
    'CCONJ': 0.07,
    'CONJ': 0.0,
    'CONTENT': 0.5769230769230769,
    'DET': 0.155,
    'EOL': 0.0,
    'FUNCTION': 0.4230769230769231,
    'INTJ': 0.01,
    'NOUN': 0.215,
    'NUM': 0.01,
    'PART': 0.0,
    'PRON': 0.035,
    'PROPN': 0.0,
    'PUNCT': 0.09,
    'SCONJ': 0.03,
    'SPACE': 0.0,
    'SYM': 0.0,
    'VERB': 0.07,
    'X': 0.0,
    'avg_ambiguation_all_words': 2.4411764705882355,
    'avg_ambiguation_content_words': 2.92,
    'avg_degree_of_abstraction': 7.193253968253968,
    'avg_rank_of_lemmas_in_freq_list': 560.565,
    'avg_sent_length': 13.333333333333334,
    'fernandez_huerta_score': 90.29653846153847,
    'logical_operator_density': 0.06951871657754011,
    'min_degree_of_abstraction': 5.0,
    'num_connectives': 8,
    'pronoun_density': 0.03626943005181347,
    'proportion_of_A_level_tokens': 0.5,
    'proportion_of_A_level_types': 0.3617021276595745,
    'syllables_per_sentence': 2